In [49]:
import numpy as np
import pandas as pd
import pathlib
import os
from tqdm.auto import tqdm


from omegaconf import DictConfig, OmegaConf

import torch
#from torch_geometric import seed_everything

import ray

In [50]:
base_path = "/home/jakobs"

project_path = f"{base_path}/data"

experiment = '231117'
experiment_path = f"{project_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

partitions = [i for i in range(22)]

## Process Predictions

In [51]:
endpoints_md = pd.read_feather(f"{base_path}/data/endpoints_md.feather").set_index("endpoint")
endpoints= endpoints_md.index.to_list()
endpoints_md

,eligable,n,freq,phecode_string,phecode_category
endpoint,,,,,
OMOP_4306655,61213,3471,0.056704,Death,Event
phecode_002,61005,535,0.008770,Staphylococcus,ID
phecode_002-1,61066,354,0.005797,Staphylococcus aureus,ID
phecode_003,61095,703,0.011507,Escherichia coli,ID
phecode_004,61049,354,0.005799,Streptococcus,ID
...,...,...,...,...,...
phecode_981,60297,1966,0.032605,Family history of malignant neoplasm,Stat
phecode_983,61178,125,0.002043,Family history of other endocrine and metaboli...,Stat
phecode_988,60364,2422,0.040123,Family history of cardiovascular disease,Stat


In [52]:
id_vars = ["eid", "model", "partition", "split"]

In [53]:
out_path = f"{experiment_path}/loghs"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

In [54]:
data_baseline = pd.read_feather(f"{base_path}/data/data_baseline_231117.feather")

In [55]:
data_outcomes_long = pd.read_feather(f"{base_path}/data/data_outcomes_long_230320.feather")

In [56]:
data_outcomes_long.query("event==1").sample(10)

,eid,endpoint,prev,event,time
2807438,RID02514,phecode_401,0.0,1.0,4.416153
2981680,RID02814,phecode_416-2,1.0,1.0,9.416381
1863242,RID00517,phecode_288,0.0,1.0,3.373032
1737847,RID05701,phecode_282-1,0.0,1.0,6.106776
2853910,RID05570,phecode_404,1.0,1.0,0.539357
4342525,RID00990,phecode_593,0.0,1.0,7.329227
3016838,RID01704,phecode_424,0.0,1.0,8.457221
401390,RID02812,phecode_101,0.0,1.0,4.848734
3876123,RID05818,phecode_522,0.0,1.0,3.901437
3120141,RID03576,phecode_438,0.0,1.0,10.893908


In [57]:
endpoint_selection = [
    # generally very important
    "phecode_202", # Diabetes mellitus
    "phecode_401",	#Hypertension"  
    "phecode_404", # Ischemic heart disease   
    "phecode_404-1", # Myocardial infarction [Heart attack]
    "phecode_431-11", # Cerebral infarction [Ischemic stroke]
    "phecode_424", # Heart failure

    
    "phecode_059-1", # COVID 19
    "phecode_468", # Pneumonia
    "phecode_474", # Chronic obstructive pulmonary disease [COPD]
      
    "phecode_286-2", #	Major depressive disorder
    "phecode_324-11", #Parkinson's Disease
    "phecode_328", # Dementias and cerebral degeneration

    
    "phecode_164", # Anemia
    "phecode_726-1", # Osteoporosis
    "phecode_371", # Cataract
    "phecode_374-42", # Diabetic retinopathy
    "phecode_374-5", # Macular degeneration
    "phecode_375-1", # Glaucoma
    
    
    "phecode_103", # Malignant neoplasm of the skin
    "phecode_101", # Malignant neoplasm of the digestive organs
    "phecode_102", # LUNG CANCER
    
    "phecode_583", # Chronic kidney disease    
    "phecode_542", # Chronic liver disease and sequelae    
    "OMOP_4306655"] # All-Cause Death

In [58]:
endpoint_frequencies = data_outcomes_long.query("event==1").value_counts("endpoint").to_frame("n_epic").merge(endpoints_md, how="left", left_index=True, right_index=True)

In [59]:
endpoint_frequencies.query("endpoint==@endpoint_selection")

,n_epic,eligable,n,freq,phecode_string,phecode_category
endpoint,,,,,,
phecode_401,2637,56016.0,16709.0,0.298290,Hypertension,Cardio
phecode_371,1209,60239.0,5752.0,0.095486,Cataract,Eye
phecode_404,969,59252.0,5543.0,0.093550,Ischemic heart disease,Cardio
phecode_103,651,60349.0,2705.0,0.044823,Malignant neoplasm of the skin,Neoplasms
phecode_164,648,59901.0,4294.0,0.071685,Anemia,Blood
phecode_202,620,59873.0,4619.0,0.077147,Diabetes mellitus,Endo
phecode_468,534,60719.0,3057.0,0.050347,Pneumonia,Resp
phecode_583,485,61045.0,2121.0,0.034745,Chronic kidney disease,Genitourinary
phecode_424,430,60962.0,1694.0,0.027788,Heart failure,Cardio


In [60]:
endpoint_frequencies.query("n_epic>20&phecode_string==phecode_string").sort_index().reset_index().to_feather(f"{base_path}/data/endpoints_epic_md.feather")

In [61]:
endpoints_md = pd.read_feather(f"{base_path}/data/endpoints_epic_md.feather").set_index("endpoint")
endpoints_md

,n_epic,eligable,n,freq,phecode_string,phecode_category
endpoint,,,,,,
OMOP_4306655,111,61213.0,3471.0,0.056704,Death,Event
phecode_052,31,61142.0,197.0,0.003222,Herpesvirus,ID
phecode_052-3,28,61175.0,121.0,0.001978,Varicella zoster virus,ID
phecode_052-32,27,61184.0,117.0,0.001912,Herpes zoster,ID
phecode_061,40,61194.0,185.0,0.003023,Influenza virus,ID
...,...,...,...,...,...,...
phecode_829,129,60763.0,1980.0,0.032586,Nonspecific findings on examination of blood,Signs/Symptoms
phecode_848,85,61196.0,143.0,0.002337,Nonspecific abnormal findings of other body st...,Signs/Symptoms
phecode_979,21,61120.0,218.0,0.003567,Transplated organ,Stat


In [62]:
data_images = pd.read_feather(f"{base_path}/data/data_images_231117.feather").rename(columns={"distfilename":"img_name"})
data_images

,eid,fundusid,img_name,side_ml,ml_glaucoma_gradability_grad,ml_glaucoma_gradability_wdiff,quality
0,RID00001,epiceye00650,0AIULA8E18C97Z2K_epiceye00650.png,Right,0.760914,0.226360,
1,RID00001,epiceye00650,0AIULA8E18C983T5_epiceye00650.png,Left,0.955591,0.043725,
2,RID00002,epiceye04756,0AIULA8E2ENQ92RP_epiceye04756.png,Left,0.986005,0.013981,
3,RID00002,epiceye04756,0AIULA8E2ENQ8YKN_epiceye04756.png,Right,0.943931,0.054606,
4,RID00003,epiceye01707,0AIULA8E149C9D03_epiceye01707.png,Right,0.981540,0.017813,
...,...,...,...,...,...,...,...
14888,RID07409,epiceye01738,0AIULA8E1ADD7WPY_epiceye01738.png,Left,0.635806,0.342445,
14889,RID07409,epiceye01738,0AIULA8E1ADD86BV_epiceye01738.png,Right,0.926931,0.072251,
14890,RID07410,epiceye05558,0AIULA8E2F62N5UZ_epiceye05558.png,Left,0.861651,0.133713,
14891,RID07411,epiceye04182,0AIULA8E1KAO83TM_epiceye04182.png,Right,0.988673,0.011106,


In [63]:
predictions = pd.read_feather(f"{base_path}/data/predictionstta_wide_231117.feather").sort_values("img_name")
predictions

,partition,img_name,iteration,OMOP_4306655,phecode_002,phecode_002-1,phecode_003,phecode_004,phecode_009,phecode_011,...,phecode_841-13,phecode_841-3,phecode_841-4,phecode_848,phecode_979,phecode_981,phecode_983,phecode_988,phecode_990,phecode_997
2273651,9,0AIULA8E0NV51BDR_epiceye07238.png,23,-3.839837,-4.439789,-4.952660,-3.144872,-0.458622,-1.884196,-1.372279,...,0.159518,-1.530105,-1.486564,-1.616636,-2.258265,-0.731873,-1.644530,-2.852647,1.137372,-3.677583
2001209,10,0AIULA8E0NV51BDR_epiceye07238.png,20,-5.339261,-5.350766,-5.023494,-5.569245,-3.101044,-4.785091,-3.358572,...,-0.881131,-3.007042,-1.806263,-1.749199,-5.537983,-0.583594,-3.416349,-2.366326,2.478284,-5.001829
783233,4,0AIULA8E0NV51BDR_epiceye07238.png,8,-6.146601,-4.466170,-4.216357,-4.203660,-2.420563,-3.820403,-2.424020,...,-2.167323,-3.430732,-4.068862,-2.571218,-3.118319,-1.142189,-1.983131,-2.525644,-0.140724,-3.836461
2594171,20,0AIULA8E0NV51BDR_epiceye07238.png,26,-3.304816,-4.553644,-4.365979,-3.222201,-1.852345,-2.818310,-1.460914,...,-1.639337,-2.177634,-2.385309,-1.993169,-3.869826,-0.024940,-0.478025,-0.911029,0.196956,-3.902972
1824923,20,0AIULA8E0NV51BDR_epiceye07238.png,18,-3.278524,-4.582914,-4.428401,-3.139032,-1.786538,-2.841114,-1.367133,...,-1.324032,-1.842844,-2.156155,-2.058772,-3.937088,0.016211,-0.427947,-0.966128,0.225033,-3.673687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30489,5,0AIULA8E33581ADU_epiceye03802.png,0,-3.668552,-4.257793,-3.856940,-2.873888,-3.007023,-3.141647,-1.938432,...,-1.503802,-2.443040,-2.307020,-2.004934,-3.092923,-0.471258,-1.944810,-2.146347,-0.056022,-2.650984
2931195,7,0AIULA8E33581ADU_epiceye03802.png,30,-3.809479,-3.411570,-3.458382,-3.409999,-2.270460,-3.197939,-2.173409,...,-1.620560,-2.088665,-1.790123,-2.195736,-2.399284,-0.831004,-1.103532,-1.773490,0.891253,-2.249069
3620313,9,0AIULA8E33581ADU_epiceye03802.png,37,-4.485674,-4.870181,-5.365415,-3.714649,-1.076989,-2.730350,-2.162106,...,0.013907,-1.791496,-1.734222,-1.807566,-2.368103,-0.778840,-2.184870,-3.033368,1.174553,-4.706765
3091455,4,0AIULA8E33581ADU_epiceye03802.png,32,-6.718625,-4.846046,-4.608363,-4.137684,-2.787123,-4.679184,-2.352429,...,-1.614635,-2.659076,-3.618710,-2.882981,-3.331972,-1.085145,-2.029951,-2.759687,-0.009096,-4.493572


In [64]:
na_count = np.isnan(predictions[endpoints].values).sum()
all_count = predictions[endpoints].values.shape[0]*predictions[endpoints].values.shape[1]
na_count/all_count

0.0

In [65]:
predictions_agg = predictions.groupby("img_name")[endpoints_md.index].mean().sort_index().reset_index()
predictions_agg

,img_name,OMOP_4306655,phecode_052,phecode_052-3,phecode_052-32,phecode_061,phecode_070,phecode_089,phecode_089-1,phecode_089-2,...,phecode_808,phecode_809,phecode_812,phecode_815,phecode_819,phecode_829,phecode_848,phecode_979,phecode_981,phecode_997
0,0AIULA8E0NV51BDR_epiceye07238.png,-3.757112,-1.627820,-0.182334,-0.069436,-2.680661,-2.444550,-3.856704,-3.685369,-3.245018,...,-2.832232,-3.272716,-3.357804,-2.176707,-4.004580,-2.053870,-1.662338,-3.085582,-0.629969,-3.146969
1,0AIULA8E0NV8EL29_epiceye03059.png,-3.936727,-1.529919,-0.238843,-0.092157,-2.581795,-2.403183,-3.901056,-3.677853,-3.339552,...,-2.754525,-3.142084,-3.246082,-2.014094,-4.089199,-2.317503,-1.828419,-2.656484,-0.505361,-2.151154
2,0AIULA8E0NV8ELNJ_epiceye03059.png,-4.141004,-1.671718,-0.286468,-0.142122,-2.869323,-2.652375,-4.112436,-3.934715,-3.509474,...,-2.950855,-3.393698,-3.587784,-2.448672,-4.295102,-2.433319,-1.986026,-2.979075,-0.568333,-2.721127
3,0AIULA8E0OXLWXKO_epiceye01658.png,-4.752013,-2.385308,-1.021998,-0.947711,-3.187404,-3.237387,-4.472214,-4.402966,-3.705159,...,-3.277483,-3.645758,-4.299125,-3.358426,-4.772653,-2.766043,-2.050182,-3.725949,-0.727569,-4.265579
4,0AIULA8E0SB0WTRT_epiceye04256.png,-3.830220,-1.858237,-0.351924,-0.255634,-2.478993,-2.464839,-3.949790,-3.797616,-3.367357,...,-2.772509,-3.277314,-3.432486,-2.439840,-4.033035,-2.269140,-1.950563,-3.363199,-0.599688,-2.623790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16021,0AIULA8E3357Z7FI_epiceye04112.png,-4.916070,-2.355493,-0.947367,-0.846339,-3.275414,-3.257792,-4.568808,-4.492426,-3.822929,...,-3.324757,-3.803386,-4.436614,-3.484133,-4.884498,-2.843724,-2.153691,-3.554188,-0.737579,-4.102893
16022,0AIULA8E33580BP5_epiceye03694.png,-4.343235,-2.277599,-0.810874,-0.744063,-2.789320,-2.907373,-4.261828,-4.160317,-3.573669,...,-3.047486,-3.484889,-3.923531,-3.033490,-4.453758,-2.601179,-2.073298,-3.786838,-0.662121,-3.557945
16023,0AIULA8E33580HKJ_epiceye03694.png,-4.197345,-2.154518,-0.735225,-0.668740,-2.812193,-2.830755,-4.140404,-4.033646,-3.427520,...,-3.023109,-3.416184,-3.814257,-2.822426,-4.339467,-2.400310,-1.880173,-3.587666,-0.694256,-3.853308
16024,0AIULA8E335815YK_epiceye03802.png,-4.160702,-1.772830,-0.236878,-0.115192,-2.727512,-2.565969,-4.067282,-3.880920,-3.443791,...,-3.070353,-3.396682,-3.641004,-2.581771,-4.325854,-2.585682,-1.938524,-3.369520,-0.597591,-3.733477


In [66]:
data_predictions_all = data_baseline[["eid"]].merge(data_images, how="left", on="eid").merge(predictions_agg, how="left", on="img_name").reset_index(drop=True)

In [67]:
data_predictions_all

,eid,fundusid,img_name,side_ml,ml_glaucoma_gradability_grad,ml_glaucoma_gradability_wdiff,quality,OMOP_4306655,phecode_052,phecode_052-3,...,phecode_808,phecode_809,phecode_812,phecode_815,phecode_819,phecode_829,phecode_848,phecode_979,phecode_981,phecode_997
0,RID00001,epiceye00650,0AIULA8E18C97Z2K_epiceye00650.png,Right,0.760914,0.226360,,-4.167503,-1.752213,-0.194549,...,-2.985861,-3.229861,-3.474761,-2.423913,-4.313069,-2.799751,-2.081054,-3.320532,-0.488607,-3.065525
1,RID00001,epiceye00650,0AIULA8E18C983T5_epiceye00650.png,Left,0.955591,0.043725,,-4.831411,-2.086286,-0.515657,...,-3.326542,-3.516933,-4.049318,-3.154398,-4.862575,-3.292347,-2.360423,-3.596535,-0.490145,-3.688621
2,RID00002,epiceye04756,0AIULA8E2ENQ92RP_epiceye04756.png,Left,0.986005,0.013981,,-4.012483,-2.081480,-0.572520,...,-2.930868,-3.050143,-3.400933,-2.369421,-4.168290,-2.616556,-2.032223,-3.653498,-0.529444,-2.882008
3,RID00002,epiceye04756,0AIULA8E2ENQ8YKN_epiceye04756.png,Right,0.943931,0.054606,,-3.215378,-1.539709,-0.033832,...,-2.461503,-2.766197,-2.739746,-1.526033,-3.488286,-1.883774,-1.456192,-2.689273,-0.608104,-2.347238
4,RID00003,epiceye01707,0AIULA8E149C9D03_epiceye01707.png,Right,0.981540,0.017813,,-4.070746,-2.107835,-0.654619,...,-2.966974,-3.351477,-3.697734,-2.688213,-4.234604,-2.339985,-1.854032,-3.531269,-0.702367,-3.665599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14887,RID07409,epiceye01738,0AIULA8E1ADD7WPY_epiceye01738.png,Left,0.635806,0.342445,,-3.877301,-1.912376,-0.535285,...,-2.663556,-3.069398,-3.353057,-2.334929,-4.015606,-2.443089,-2.094726,-3.181338,-0.539905,-1.856190
14888,RID07409,epiceye01738,0AIULA8E1ADD86BV_epiceye01738.png,Right,0.926931,0.072251,,-3.518790,-1.773029,-0.277323,...,-2.562722,-2.914084,-3.058528,-1.996465,-3.734130,-2.191627,-1.845119,-3.055484,-0.595020,-2.076473
14889,RID07410,epiceye05558,0AIULA8E2F62N5UZ_epiceye05558.png,Left,0.861651,0.133713,,-3.351393,-1.510045,-0.075772,...,-2.504061,-2.811323,-2.785999,-1.457996,-3.611397,-1.989613,-1.439201,-2.514905,-0.575957,-2.387417
14890,RID07411,epiceye04182,0AIULA8E1KAO83TM_epiceye04182.png,Right,0.988673,0.011106,,-4.015279,-1.854039,-0.277041,...,-2.935517,-3.089273,-3.343857,-2.337065,-4.175601,-2.736476,-2.053944,-3.665605,-0.440252,-3.004114


In [68]:
data_predictions_qc = data_predictions_all.query("quality!='Poor'").groupby("eid")[endpoints_md.index].mean()

In [69]:
splits = pd.read_feather(f"{base_path}/data/splits_230321.feather")

In [70]:
predictions_clean = splits.merge(data_predictions_qc, how="left", left_on="eid", right_index=True)

In [71]:
predictions_clean

,partition,split,eid,OMOP_4306655,phecode_052,phecode_052-3,phecode_052-32,phecode_061,phecode_070,phecode_089,...,phecode_808,phecode_809,phecode_812,phecode_815,phecode_819,phecode_829,phecode_848,phecode_979,phecode_981,phecode_997
0,0,test,RID00011,-4.080204,-1.945271,-0.500590,-0.387095,-2.754335,-2.697217,-4.110201,...,-2.957623,-3.388180,-3.644334,-2.571106,-4.266399,-2.380134,-1.952340,-3.319843,-0.651450,-3.089010
1,0,test,RID00013,-4.404331,-2.238611,-0.811129,-0.711111,-2.926048,-2.985175,-4.296083,...,-3.179451,-3.417987,-3.891032,-2.842741,-4.523358,-2.623850,-1.992423,-3.665021,-0.653266,-3.826646
2,0,test,RID00019,-3.915521,-1.786600,-0.386244,-0.278539,-2.759664,-2.608173,-3.995951,...,-2.861610,-3.362923,-3.527280,-2.378633,-4.123182,-2.166018,-1.797845,-3.059764,-0.679349,-3.001246
3,0,test,RID00028,-3.024825,-1.426514,0.091117,0.151317,-1.742547,-1.725308,-3.284230,...,-2.357333,-2.662623,-2.562347,-1.386491,-3.318871,-1.785287,-1.286090,-2.543743,-0.621647,-2.813918
4,0,test,RID00061,-4.005068,-1.786176,-0.295558,-0.187812,-2.727435,-2.557674,-4.027246,...,-2.925665,-3.408012,-3.593844,-2.536161,-4.195602,-2.320325,-1.879097,-3.379689,-0.610742,-3.225412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72475,9,train,RID07407,-4.102671,-1.902132,-0.471075,-0.362838,-2.719356,-2.667745,-4.123377,...,-2.868153,-3.426661,-3.687542,-2.673159,-4.260269,-2.427133,-2.082455,-3.326960,-0.614488,-2.645076
72476,9,train,RID07408,-4.662561,-2.396589,-0.948115,-0.846250,-2.914970,-3.122247,-4.433129,...,-3.273982,-3.370137,-3.994228,-3.017556,-4.717837,-2.995834,-2.195433,-3.755722,-0.602659,-3.727751
72477,9,train,RID07409,-3.698045,-1.842702,-0.406304,-0.305241,-2.168226,-2.305658,-3.821035,...,-2.613139,-2.991741,-3.205792,-2.165697,-3.874868,-2.317358,-1.969922,-3.118411,-0.567462,-1.966331
72478,9,train,RID07410,-3.351393,-1.510045,-0.075772,0.049492,-2.015144,-2.004821,-3.517985,...,-2.504061,-2.811323,-2.785999,-1.457996,-3.611397,-1.989613,-1.439201,-2.514905,-0.575957,-2.387417


In [33]:
predictions_clean.query("split=='test'").query("OMOP_4306655!=OMOP_4306655")#.isna().sum()# > 0).sum()

,partition,split,eid,OMOP_4306655,phecode_052,phecode_052-3,phecode_052-32,phecode_061,phecode_070,phecode_089,...,phecode_808,phecode_809,phecode_812,phecode_815,phecode_819,phecode_829,phecode_848,phecode_979,phecode_981,phecode_997
44191,6,test,RID06876,NaN,-2.007028,78.697235,-2.139163,136.340317,-2.185052,414.199005,...,-2.167738,-1397.289673,-2.097943,-2.020973,-449.929108,-1.070768,-144.791962,-2.036820,-461.818054,-76.202057
50802,7,test,RID00625,NaN,-2.040062,-746.512207,-2.172519,-182.423767,-2.210280,-119.100716,...,-2.202129,519.212280,-2.118412,-2.065345,455.834564,-1.461121,261.282776,-2.073011,-1694.987549,462.811157


In [72]:
model = "RetinaUKB"

for partition in range(10):
    temp_partition = predictions_clean.query("partition==@partition")
    for split in ["train", "test"]: 
        fp_out = f"{out_path}/{model}/{partition}"
        pathlib.Path(fp_out).mkdir(parents=True, exist_ok=True)
        t = temp_partition.query("split==@split").reset_index(drop=True)
        t.to_feather(f"{fp_out}/{split}.feather")
        print(f"{fp_out}/{split}.feather")

/home/jakobs/data/231117/loghs/RetinaUKB/0/train.feather
/home/jakobs/data/231117/loghs/RetinaUKB/0/test.feather
/home/jakobs/data/231117/loghs/RetinaUKB/1/train.feather
/home/jakobs/data/231117/loghs/RetinaUKB/1/test.feather
/home/jakobs/data/231117/loghs/RetinaUKB/2/train.feather
/home/jakobs/data/231117/loghs/RetinaUKB/2/test.feather
/home/jakobs/data/231117/loghs/RetinaUKB/3/train.feather
/home/jakobs/data/231117/loghs/RetinaUKB/3/test.feather
/home/jakobs/data/231117/loghs/RetinaUKB/4/train.feather
/home/jakobs/data/231117/loghs/RetinaUKB/4/test.feather
/home/jakobs/data/231117/loghs/RetinaUKB/5/train.feather
/home/jakobs/data/231117/loghs/RetinaUKB/5/test.feather
/home/jakobs/data/231117/loghs/RetinaUKB/6/train.feather
/home/jakobs/data/231117/loghs/RetinaUKB/6/test.feather
/home/jakobs/data/231117/loghs/RetinaUKB/7/train.feather
/home/jakobs/data/231117/loghs/RetinaUKB/7/test.feather
/home/jakobs/data/231117/loghs/RetinaUKB/8/train.feather
/home/jakobs/data/231117/loghs/RetinaUK

In [134]:
out_path

'/home/jakobs/data/230321/loghs'

In [20]:
1+1

2